In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [18]:

# Carregue seus dados em um dataframe (supondo que o nome do dataframe seja "dados")
dados = pd.read_csv("data/train_with_flight_v2.csv")
dados = dados[dados["target"] > 300]

In [19]:
dados["sum(flight_through_area)"].max()

446.0

In [20]:
del dados["HoraData"],dados["HoraDataDest"]

In [21]:
del dados["flightid_"]

In [22]:
del dados["trocareal"],dados["troca_prev_mean_x"],dados["troca_prev_sum_x"],dados["troca_prev_mean_y"],dados["troca_prev_sum_y"],dados["troca_prev_mean"],dados["troca_prev_sum"],dados["troca_real_x"],dados["troca_real_y"],dados["troca_real"]

In [23]:
# Primeiro, vamos carregar os valores únicos a partir do arquivo CSV
origem = pd.read_csv('data/origem.csv')
mapping_dict = {i: val for i, val in enumerate(origem.values)}
# Agora, vamos reverter os valores codificados de volta aos valores originais
dados['origem'] = dados['origem'].map(mapping_dict)

destino = pd.read_csv('data/destino.csv')
mapping_dict = {i: val for i, val in enumerate(destino.values)}

dados['destino'] = dados['destino'].map(mapping_dict)


In [24]:
dados = dados[dados["origem"] != dados["destino"]]

In [25]:
dados["origem"] = dados["origem"].astype('str')
dados["destino"] = dados["destino"].astype('str')

In [26]:
dados["origem"], origem = pd.factorize(dados["origem"]) #verificar se rodando novamente muda de código
dados["destino"], destino = pd.factorize(dados["destino"])


In [27]:
for col in dados.columns:
    print(col)

flightid
origem
destino
target
linha
diaSemana
hora
esperas
mean_x
sum_x
mean_y
sum_y
mean
sum
Temperaturamean_x
Temperaturamin_x
Temperaturamax_x
Temperaturastd_x
Ponto de Orvalhomean_x
Ponto de Orvalhomin_x
Ponto de Orvalhomax_x
Ponto de Orvalhostd_x
Velocidade do Ventomean_x
Velocidade do Ventomin_x
Velocidade do Ventomax_x
Velocidade do Ventostd_x
Direção do Ventomean_x
Direção do Ventomin_x
Direção do Ventomax_x
Direção do Ventostd_x
Visibilidademean_x
Visibilidademin_x
Visibilidademax_x
Visibilidadestd_x
Pressãomean_x
Pressãomin_x
Pressãomax_x
Pressãostd_x
Temperaturamean_y
Temperaturamin_y
Temperaturamax_y
Temperaturastd_y
Ponto de Orvalhomean_y
Ponto de Orvalhomin_y
Ponto de Orvalhomax_y
Ponto de Orvalhostd_y
Velocidade do Ventomean_y
Velocidade do Ventomin_y
Velocidade do Ventomax_y
Velocidade do Ventostd_y
Direção do Ventomean_y
Direção do Ventomin_y
Direção do Ventomax_y
Direção do Ventostd_y
Visibilidademean_y
Visibilidademin_y
Visibilidademax_y
Visibilidadestd_y
Pressãomea

In [28]:
#del dados["troca_real"], dados["troca_realtcreal_sem"], dados["troca_realb5"]


In [29]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
X = dados.drop(["target", "flightid"], axis=1)
y = dados["target"]
sc = StandardScaler()
orderCols = X.columns
X = sc.fit_transform(X)

test = pd.read_csv("data/test_gambi.csv")
#del test["troca_prev_mean"], test["troca_prev_sum"]
del test["hora_ref"], test["dt_dep"], test["snapshot_radar"], test["path"], test["hora_esperas"], test["aero_esperas"], test["hora_metaf"], test["metaf"], test["aero_metaf"], test["hora_metar"], test["metar"], test["aero_metar"], test["hora_tcp"], test["aero_tcp"], test["hora_tcr"], test["aero_tcr"], test["eqq"], test["data"], test["first(substring(CAST(polygon AS STRING) 3 13))"], test["flight_through_area"], test["aero_x"], test["aero_y"], test["aero"], test["HoraData"], test["linha_first"], test["flightid_"]
test["sum(flight_through_area)"] = test["sum(flight_through_area)"]/446
aux = np.where(test["origem"] == test["destino"], 1, 0)
del test["flightid"]
origem = pd.read_csv('data/origem.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(origem.values)}
# Primeiro, vamos carregar os valores únicos a partir do arquivo CSV

# Agora, vamos reverter os valores codificados de volta aos valores originais
test['origem'] = test['origem'].map(mapping_dict)

destino = pd.read_csv('data/destino.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(destino.values)}

test['destino'] = test['destino'].map(mapping_dict)

destino = pd.read_csv('data/linha.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(destino.values)}

test['linha'] = test['linha'].map(mapping_dict)

#orderCols = ["origem","destino","linha","diaSemana","hora","esperas","mean_x","sum_x","mean_y","sum_y","mean","sum","Temperaturamean_x","Temperaturamin_x","Temperaturamax_x","Temperaturastd_x","Ponto de Orvalhomean_x","Ponto de Orvalhomin_x","Ponto de Orvalhomax_x","Ponto de Orvalhostd_x","Velocidade do Ventomean_x","Velocidade do Ventomin_x","Velocidade do Ventomax_x","Velocidade do Ventostd_x","Direção do Ventomean_x","Direção do Ventomin_x","Direção do Ventomax_x","Direção do Ventostd_x","Visibilidademean_x","Visibilidademin_x","Visibilidademax_x","Visibilidadestd_x","Pressãomean_x","Pressãomin_x","Pressãomax_x","Pressãostd_x","Temperaturamean_y","Temperaturamin_y","Temperaturamax_y","Temperaturastd_y","Ponto de Orvalhomean_y","Ponto de Orvalhomin_y","Ponto de Orvalhomax_y","Ponto de Orvalhostd_y","Velocidade do Ventomean_y","Velocidade do Ventomin_y","Velocidade do Ventomax_y","Velocidade do Ventostd_y","Direção do Ventomean_y","Direção do Ventomin_y","Direção do Ventomax_y","Direção do Ventostd_y","Visibilidademean_y","Visibilidademin_y","Visibilidademax_y","Visibilidadestd_y","Pressãomean_y","Pressãomin_y","Pressãomax_y","Pressãostd_y","troca","dt_radar_lambda","dt_radar_lambda1","dt_radar_lambda2","dt_radar_lambda3","flightlevel_mean","flightlevel_max","distancia_","flightlevel_std","flightlevel_std1","speed_std","speed_std1", "sum(flight_through_area)"]

#y_test = model.predict(test)
test = sc.transform(test[orderCols])
#y_test = model.predict(test, num_iteration=model.best_iteration)



In [30]:
nfolds = 10
kf = KFold(n_splits=nfolds, shuffle=True, random_state=42) # you can change 'n_splits' to define how many folds you want

auxtest = pd.DataFrame()

auxtest["ID"] = pd.read_csv("data/test_with_flight_v2.csv")["flightid"]
auxtest["solution"] = 0
mse_list = []

for train_indices, test_indices in kf.split(X):
    X_train, y_train = X[train_indices], y.iloc[train_indices]
    X_test, y_test =   X[test_indices],  y.iloc[test_indices]

    d_train = lgb.Dataset(X_train, label=y_train)

    params = {'objective': 'regression',
              'metric': "rmse",  # try rmse
              'num_leaves': 31,
              'learning_rate': 0.05,
              'feature_fraction': 0.9,
              'bagging_fraction': 0.8,
              'bagging_freq': 5}

    model = lgb.train(params, d_train, num_boost_round=1000, valid_sets=[d_train],
                      callbacks=([lgb.early_stopping(stopping_rounds=10)]))

    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mse_list.append(rmse)
    print(f"RMSE: {rmse}")
    y_test = model.predict(test, num_iteration=model.best_iteration)
    auxtest["solution"] = auxtest["solution"] + y_test
    
auxtest["solution"] = auxtest["solution"] / nfolds
auxtest["solution"] = np.where(aux == 1, 0, auxtest["solution"])
auxtest[['ID', 'solution']].to_csv('data/submission_flight_through_area_10folds_g1500.csv', index=False)
print("Mean RMSE: ", np.mean(mse_list))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3518
[LightGBM] [Info] Number of data points in the train set: 52849, number of used features: 73
[LightGBM] [Info] Start training from score 8365.430623


/home/bruno/.local/lib/python3.10/site-packages/lightgbm/callback.py:294: UserWarning: Only training set found, disabling early stopping.
  _log_warning('Only training set found, disabling early stopping.')


RMSE: 642.0305436608469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3529
[LightGBM] [Info] Number of data points in the train set: 52849, number of used features: 73
[LightGBM] [Info] Start training from score 8373.280781


/home/bruno/.local/lib/python3.10/site-packages/lightgbm/callback.py:294: UserWarning: Only training set found, disabling early stopping.
  _log_warning('Only training set found, disabling early stopping.')


RMSE: 411.3827467320571
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003053 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3542
[LightGBM] [Info] Number of data points in the train set: 52850, number of used features: 73
[LightGBM] [Info] Start training from score 8371.269063


...

RMSE: 385.2536273296889
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003957 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3527
[LightGBM] [Info] Number of data points in the train set: 52850, number of used features: 73
[LightGBM] [Info] Start training from score 8368.655572


/home/bruno/.local/lib/python3.10/site-packages/lightgbm/callback.py:294: UserWarning: Only training set found, disabling early stopping.
  _log_warning('Only training set found, disabling early stopping.')


RMSE: 413.178911888419
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007465 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3523
[LightGBM] [Info] Number of data points in the train set: 52850, number of used features: 73
[LightGBM] [Info] Start training from score 8369.080057


/home/bruno/.local/lib/python3.10/site-packages/lightgbm/callback.py:294: UserWarning: Only training set found, disabling early stopping.
  _log_warning('Only training set found, disabling early stopping.')


RMSE: 457.43874810561294
Mean RMSE:  503.2560867247372


In [31]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Divide the data into features (X) and target column (y)
X = dados.drop(["target", "flightid"], axis=1)
y = dados["target"]
# Find columns with 4 or fewer unique values in X
cols_to_transform = [col for col in X.columns if X[col].nunique() <= 4]

# Apply get_dummies on these columns
#X = pd.get_dummies(X, columns=cols_to_transform)
# Divide the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A good practice is standardizing the features for better model performance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Create a LightGBM Dataset object for better performance
d_train = lgb.Dataset(X_train, label=y_train)

# Define the parameters of the LightGBM model
params = {    'objective': 'regression',
    'metric': "rmse",  # tentar rmse
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5}

# Train the LightGBM model with early stopping
model = lgb.train(params, d_train, num_boost_round=1000, valid_sets=[d_train], callbacks=(
            [lgb.early_stopping(stopping_rounds=10)]))

# Make predictions on the test set using the best iteration
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Calculate the root mean square error (RMSE) in the predictions
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3481
[LightGBM] [Info] Number of data points in the train set: 46977, number of used features: 73
[LightGBM] [Info] Start training from score 8369.903676


/home/bruno/.local/lib/python3.10/site-packages/lightgbm/callback.py:294: UserWarning: Only training set found, disabling early stopping.
  _log_warning('Only training set found, disabling early stopping.')


RMSE: 540.1985721039136


In [32]:
ref = pd.read_csv("data/refactorSub.csv")
ref["solution"].corr(auxtest["solution"])

-0.17768095334911124

In [29]:
for col in test.columns:
    if col not in X.columns:
        print(col)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
X.shape

(285960, 73)

In [31]:
del test["flightid"]

In [32]:
origem = pd.read_csv('data/origem.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(origem.values)}

In [33]:
# Primeiro, vamos carregar os valores únicos a partir do arquivo CSV

# Agora, vamos reverter os valores codificados de volta aos valores originais
test['origem'] = test['origem'].map(mapping_dict)


In [34]:
test

,origem,destino,esperas,troca,diaSemana,linha,hora,mean_x,sum_x,mean_y,...,dt_radar_lambda2,dt_radar_lambda3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std1,speed_std,speed_std1,sum(flight_through_area)
0,2,SBSP,1,0.0,0.0,SBBRSBSP,11.0,0.000000,0.0,0.009058,...,10564.0,844.206687,269.510868,372.424169,834.699654,19.009494,114.817624,87.142962,20.135674,0.982063
1,4,SBCT,0,0.0,4.0,SBSPSBCT,0.0,0.654321,212.0,0.486702,...,7381.0,334.871433,164.239133,276.821248,334.254521,9.689498,87.120708,81.325769,14.762520,0.392377
2,1,SBPA,0,0.0,6.0,SBGRSBPA,1.0,0.070988,23.0,0.210145,...,9122.0,722.301597,254.844834,351.092908,865.298618,17.465243,116.247042,76.296121,15.923236,1.383408
3,7,SBSV,0,0.0,4.0,SBCTSBSV,3.0,0.006173,2.0,0.032801,...,8944.0,552.370292,328.999659,383.333333,1752.457895,23.451966,93.777156,67.783456,21.149632,0.921525
4,5,SBGR,0,0.0,2.0,SBRJSBGR,0.0,0.623457,202.0,0.212862,...,13445.0,423.794971,170.393562,291.021939,344.671385,9.565269,93.363084,76.759758,12.799802,0.582960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40034,10,SBRF,0,0.0,0.0,SBSVSBRF,20.0,0.000000,0.0,0.009964,...,9182.0,526.762331,226.645097,362.508384,643.117568,15.707042,127.343373,97.043441,14.564714,0.809417
40035,7,SBSP,0,0.0,5.0,SBCTSBSP,8.0,0.006173,2.0,0.022645,...,6902.0,490.297328,165.953869,276.223053,335.494072,10.369936,86.372285,96.123926,16.439044,0.233184
40036,1,SBCF,0,0.0,1.0,SBGRSBCF,11.0,0.314815,102.0,0.329710,...,7620.0,452.974918,201.908039,332.682734,483.495187,12.614499,103.607766,95.569591,15.688727,0.511211
40037,2,SBGR,0,0.0,2.0,SBBRSBGR,12.0,0.009259,3.0,0.034420,...,15064.0,636.873657,263.464528,366.306658,824.563068,16.204102,112.596668,84.884888,17.952150,0.573991


In [35]:

destino = pd.read_csv('data/destino.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(destino.values)}

test['destino'] = test['destino'].map(mapping_dict)


In [36]:
orderCols = ["origem","destino","linha","diaSemana","hora","esperas","mean_x","sum_x","mean_y","sum_y","mean","sum","Temperaturamean_x","Temperaturamin_x","Temperaturamax_x","Temperaturastd_x","Ponto de Orvalhomean_x","Ponto de Orvalhomin_x","Ponto de Orvalhomax_x","Ponto de Orvalhostd_x","Velocidade do Ventomean_x","Velocidade do Ventomin_x","Velocidade do Ventomax_x","Velocidade do Ventostd_x","Direção do Ventomean_x","Direção do Ventomin_x","Direção do Ventomax_x","Direção do Ventostd_x","Visibilidademean_x","Visibilidademin_x","Visibilidademax_x","Visibilidadestd_x","Pressãomean_x","Pressãomin_x","Pressãomax_x","Pressãostd_x","Temperaturamean_y","Temperaturamin_y","Temperaturamax_y","Temperaturastd_y","Ponto de Orvalhomean_y","Ponto de Orvalhomin_y","Ponto de Orvalhomax_y","Ponto de Orvalhostd_y","Velocidade do Ventomean_y","Velocidade do Ventomin_y","Velocidade do Ventomax_y","Velocidade do Ventostd_y","Direção do Ventomean_y","Direção do Ventomin_y","Direção do Ventomax_y","Direção do Ventostd_y","Visibilidademean_y","Visibilidademin_y","Visibilidademax_y","Visibilidadestd_y","Pressãomean_y","Pressãomin_y","Pressãomax_y","Pressãostd_y","troca","dt_radar_lambda","dt_radar_lambda1","dt_radar_lambda2","dt_radar_lambda3","flightlevel_mean","flightlevel_max","distancia_","flightlevel_std","flightlevel_std1","speed_std","speed_std1", "sum(flight_through_area)"]


In [37]:

destino = pd.read_csv('data/linha.csv')
mapping_dict = {str(val)[2:-2]: i for i, val in enumerate(destino.values)}

test['linha'] = test['linha'].map(mapping_dict)


In [38]:
#y_test = model.predict(test)
test = sc.transform(test[orderCols])
y_test = model.predict(test, num_iteration=model.best_iteration)

In [71]:
auxtest = pd.DataFrame()
auxtest["ID"] = pd.read_csv("data/test_with_flight_v2.csv")["flightid"]
auxtest["solution"] = y_test
auxtest["solution"] = np.where(aux == 1, 0, auxtest["solution"])
auxtest[['ID', 'solution']].to_csv('data/submission_flight_through_area.csv', index=False)

In [167]:
feature_importance = model.feature_importance()

# Para melhor visualização, você pode formar um DataFrame pandas
importance_df = pd.DataFrame({
    'feature': model.feature_name(),
    'importance': feature_importance
}).sort_values(by='importance', ascending=False)

In [169]:
X

,origem,destino,linha,diaSemana,hora,esperas,mean_x,sum_x,mean_y,sum_y,...,dt_radar_lambda1,dt_radar_lambda2,dt_radar_lambda3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std1,speed_std,speed_std1
0,0,0,0,2,0,0.0,0.694444,225,0.221014,244,...,4.0,7565.0,521.892427,155.444305,278.150593,405.268959,10.629109,94.311322,95.621127,16.187571
1,1,0,1,2,0,0.0,0.987654,320,0.254529,281,...,0.0,8344.0,494.427290,132.958313,248.328512,343.265933,9.612205,81.397118,91.296811,16.387080
2,2,0,2,2,0,0.0,0.000000,0,0.034420,38,...,4.0,9237.0,595.135591,262.040541,381.191661,910.308038,14.652328,130.701604,95.461995,17.738672
3,0,1,3,2,0,0.0,0.694444,225,0.221014,244,...,0.0,9063.0,524.253328,196.231226,319.851729,487.833173,15.777718,101.901545,94.390050,17.728770
4,3,2,4,2,1,1.0,0.055556,18,0.173007,191,...,1801.0,7383.0,444.785599,260.408845,348.959502,998.072220,15.047433,112.341484,79.346316,13.189539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286075,2,7,23,5,14,0.0,0.015432,5,0.004529,5,...,4.0,10564.0,849.002842,270.399060,373.004778,834.450561,19.223059,114.671695,86.855591,20.304259
286076,9,7,27,5,14,0.0,0.027778,9,0.015399,17,...,0.0,13261.0,627.347120,252.206026,377.414083,831.091806,12.165105,126.147031,107.373468,14.994331
286077,7,7,22,5,15,0.0,0.046296,15,0.022645,25,...,1.0,10624.0,499.394442,166.025909,276.126944,335.394812,10.307363,86.371366,96.211542,16.216914
286078,5,7,24,5,15,0.0,0.243827,79,0.160326,177,...,0.0,12603.0,513.424001,173.087082,293.534407,371.187610,10.346388,94.731864,81.768601,13.809341


In [168]:
importance_df

,feature,importance
4,Column_4,1903
5,Column_5,1269
6,Column_6,1125
24,Column_24,1121
32,Column_32,1072
...,...,...
43,Column_43,1
59,Column_59,0
55,Column_55,0
51,Column_51,0


In [65]:
len(y_test)

40039

In [63]:
y_test

array([ 14668.95872363, -38577.11842178, -36857.02373341, ...,
        20588.97186654, -37318.52885762,  19701.28586337])

In [15]:
cols_to_transform

['troca', 'troca_prev_mean', 'troca_prev_sum']

In [14]:
dados

,flightid,origem,destino,target,linha,diaSemana,hora,esperas,mean_x,sum_x,...,dt_radar_lambdamax,dt_radar_lambdastd,flightlevel_meanmean,flightlevel_maxmean,distancia_mean,flightlevel_stdstd,flightlevel_stdmean,speed_stdmean,speed_stdstd,sum(flight_through_area)
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,2,0,0,0.694444,225,...,7565.0,534.960010,155.344886,278.423923,405.469774,10.830585,94.289190,95.582598,16.348629,0.0
1,c7c5c10716335b048f86d8c52fcba3f2,1,0,2152,1,2,0,0,0.987654,320,...,8344.0,507.802997,132.867278,248.432199,343.316900,9.681570,81.367359,91.239395,16.384289,0.0
2,3162de7203a972f071d5a48e8f0f4828,2,0,4613,2,2,0,0,0.000000,0,...,9237.0,613.391446,261.703663,380.862522,909.958818,14.824041,130.649077,95.292437,17.765935,0.0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,1,2581,3,2,0,0,0.694444,225,...,9063.0,531.192062,196.294996,320.127790,487.733487,15.784746,101.979232,94.354543,17.811656,0.0
4,504a62621cd231d6ab67e674ce538cd3,3,2,5547,4,2,1,1,0.055556,18,...,7383.0,455.099625,260.850332,349.653333,997.430556,14.632192,112.581055,79.572741,13.370615,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,1,0,2138,1,5,21,0,0.123457,40,...,8344.0,507.802997,132.867278,248.432199,343.316900,9.681570,81.367359,91.239395,16.384289,0.0
262412,c1e1fcbfd1d1c67c01da384ad104c0df,2,0,4741,2,5,21,0,0.074074,24,...,9237.0,613.391446,261.703663,380.862522,909.958818,14.824041,130.649077,95.292437,17.765935,0.0
262413,612af5b2b7cc48161bee87965785282b,6,0,3669,37,5,21,0,0.033951,11,...,9423.0,569.084851,241.221515,378.838369,751.804286,12.110501,133.070262,109.174977,15.306432,0.0
262414,5efa9c4eb13e214f29a73a82cf7822e4,10,0,7192,38,5,12,0,0.000000,0,...,10924.0,630.301570,291.511873,378.088957,1225.918107,12.702017,123.871406,82.845625,13.215120,0.0


In [18]:


# Divida os dados em recursos/features (X) e coluna alvo/target (y)
X = dados.drop(["target", "flightid"], axis=1)
y = dados["target"]

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie um objeto DMatrix para o XGBoost para melhor desempenho
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Defina os parâmetros do modelo XGBoost
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.15,
    "max_depth": 3,
    "gamma": 5
    
}

# Treine o modelo XGBoost
model = xgb.train(params, dtrain, num_boost_round=100)

# Faça previsões no conjunto de teste
y_pred = model.predict(dtest)

# Calcule o erro da raiz quadrada média (RMSE) nas previsões
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")


/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

RMSE: 792.3648567855787


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
from sklearn import metrics
dados.fillna(-99, inplace=True)
# Divida os dados em recursos/features (X) e coluna alvo/target (y)
X = dados.drop(["target", "flightid"], axis=1)
y = dados["target"]

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hiperparâmetros
xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.15,
    "max_depth": 3,
    "gamma": 5
}

lgb_params = {
    'objective': 'regression',
    'metric': ['l2', 'l1'],  # Alterado para uma lista
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5
}


# Defina os regressores de base
regressor1 = lgb.LGBMRegressor(**lgb_params)  # LightGBM como regressor de base
regressor2 = xgb.XGBRegressor(**xgb_params)  # XGBoost como regressor de base
regressor3 = RandomForestRegressor(random_state=1)

# Defina o meta-regressor
stacked_regressor = RandomForestRegressor(random_state=1) 

# Stacking dos regressores
stacked = StackingCVRegressor(regressors=(regressor1, regressor2, regressor3),
                              meta_regressor=stacked_regressor, random_state=42)

# Treinamento dos modelos
for regressor, label in zip([regressor1, regressor2, regressor3, stacked], 
                            ['LightGBM', 
                             'XGBoost', 
                             'Random Forest',
                             'Stacked Regressor']):
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print(f'RMSE [{label}]: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.148658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10005
[LightGBM] [Info] Number 

/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

RMSE [XGBoost]: 792.36
RMSE [Random Forest]: 683.83
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total 

/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

RMSE [Stacked Regressor]: 853.77


In [16]:
from joblib import dump

# Salvar o modelo em disco
dump(stacked, 'stacked_model.joblib')


['stacked_model.joblib']

In [1]:
from joblib import load

# load the model from disk
model = load('stacked_model.joblib')


In [3]:
import pandas as pd
test = pd.read_csv("data/idsc_test.csv")

In [7]:
dados

,flightid,origem,destino,target,linha,diaSemana,hora,HoraDataDest,HoraData,esperas,...,dt_radar_lambdamax,dt_radar_lambdastd,flightlevel_meanmean,flightlevel_maxmean,distancia_mean,flightlevel_stdstd,flightlevel_stdmean,speed_stdmean,speed_stdstd,sum(flight_through_area)
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,2,0,2022-06-01 01,2022-06-01 00,0,...,7565.0,534.960010,155.344886,278.423923,405.469774,10.830585,94.289190,95.582598,16.348629,0.0
1,c7c5c10716335b048f86d8c52fcba3f2,1,0,2152,1,2,0,2022-06-01 01,2022-06-01 00,0,...,8344.0,507.802997,132.867278,248.432199,343.316900,9.681570,81.367359,91.239395,16.384289,0.0
2,3162de7203a972f071d5a48e8f0f4828,2,0,4613,2,2,0,2022-06-01 01,2022-06-01 00,0,...,9237.0,613.391446,261.703663,380.862522,909.958818,14.824041,130.649077,95.292437,17.765935,0.0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,1,2581,3,2,0,2022-06-01 01,2022-06-01 00,0,...,9063.0,531.192062,196.294996,320.127790,487.733487,15.784746,101.979232,94.354543,17.811656,0.0
4,504a62621cd231d6ab67e674ce538cd3,3,2,5547,4,2,1,2022-06-01 02,2022-06-01 01,1,...,7383.0,455.099625,260.850332,349.653333,997.430556,14.632192,112.581055,79.572741,13.370615,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,1,0,2138,1,5,21,2023-04-15 22,2023-04-15 21,0,...,8344.0,507.802997,132.867278,248.432199,343.316900,9.681570,81.367359,91.239395,16.384289,0.0
262412,c1e1fcbfd1d1c67c01da384ad104c0df,2,0,4741,2,5,21,2023-04-15 22,2023-04-15 21,0,...,9237.0,613.391446,261.703663,380.862522,909.958818,14.824041,130.649077,95.292437,17.765935,0.0
262413,612af5b2b7cc48161bee87965785282b,6,0,3669,37,5,21,2023-04-15 22,2023-04-15 21,0,...,9423.0,569.084851,241.221515,378.838369,751.804286,12.110501,133.070262,109.174977,15.306432,0.0
262414,5efa9c4eb13e214f29a73a82cf7822e4,10,0,7192,38,5,12,2023-04-15 14,2023-04-15 12,0,...,10924.0,630.301570,291.511873,378.088957,1225.918107,12.702017,123.871406,82.845625,13.215120,0.0


In [9]:
import pandas as pd
from metar.Metar import Metar

# Função para parsear METAR
def parse_metar(metar):
    metar = metar.replace('METAF', 'METAR') 
    try:
        metar_obj = Metar(metar)
        temperature = metar_obj.temp.value() if metar_obj.temp else None
        dewpoint = metar_obj.dewpt.value() if metar_obj.dewpt else None
        wind_speed = metar_obj.wind_speed.value() if metar_obj.wind_speed else None
        wind_direction = metar_obj.wind_dir.value() if metar_obj.wind_dir else None
        visibility = metar_obj.vis.value() if metar_obj.vis else None
        pressure = metar_obj.press.value() if metar_obj.press else None
        weather = ', '.join([str(c) for c in metar_obj.weather]) if metar_obj.weather else None
        sky = ', '.join([str(c) for c in metar_obj.sky]) if metar_obj.sky else None
        return pd.Series([temperature, dewpoint, wind_speed, wind_direction, visibility, pressure, weather, sky])
    except Exception as e:
        #print(str(e))
        return pd.Series([None, None, None, None, None, None, None, None])

# Aplicar parse sobre METAR para cada linha
test[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = test['metar'].apply(parse_metar)



/tmp/ipykernel_76367/1068738982.py:23: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  test[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = test['metar'].apply(parse_metar)


In [10]:
test

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,path,hora_esperas,esperas,aero_esperas,...,hora_tcr,aero_tcr,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento,Visibilidade,Pressão,Tempo,Céu
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,2023-05-30 11:36:03.000,2023-05-30 11:00:00.000,MULTIPOINT ((-0.8807200448127578 -0.5148453155...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-30 10:00:00.000,0,SBSV,...,NaN,NaN,28.0,21.0,9.0,120.0,10000.0,1019.0,None,"('FEW', <metar.Datatypes.distance object at 0x..."
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,2023-05-19 08:26:06.000,2023-05-19 08:00:00.000,MULTIPOINT ((-0.8826266039383116 -0.4926305156...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 07:00:00.000,0,SBRJ,...,NaN,NaN,19.0,17.0,4.0,20.0,10000.0,1019.0,None,"('FEW', <metar.Datatypes.distance object at 0x..."
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,2023-05-22 11:52:02.000,2023-05-22 11:00:00.000,MULTIPOINT ((-0.8972512464147154 -0.5215196550...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-22 10:00:00.000,0,SBGR,...,NaN,NaN,13.0,12.0,6.0,70.0,10000.0,1022.0,NaN,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2023-05-29 16:07:07.000,2023-05-29 16:00:00.000,MULTIPOINT ((-0.892972783728755 -0.51704889771...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-29 15:00:00.000,2,SBSP,...,NaN,NaN,19.0,14.0,10.0,120.0,10000.0,1023.0,None,"('SCT', <metar.Datatypes.distance object at 0x..."
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,2023-05-25 22:16:04.000,2023-05-25 22:00:00.000,MULTIPOINT ((-0.8970045369724385 -0.5240544927...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-25 21:00:00.000,0,SBKP,...,NaN,NaN,23.0,13.0,4.0,90.0,10000.0,1020.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6f6d576aafbd90e308a848a060395e9a,SBPA,SBGR,2023-05-19 22:02:03.000,2023-05-19 22:00:00.000,MULTIPOINT ((-0.8891070333352145 -0.5128856751...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 21:00:00.000,0,SBGR,...,NaN,NaN,16.0,13.0,3.0,140.0,10000.0,1021.0,NaN,NaN
96,20055ed6ecea5e916923081fc3b049f7,SBCF,SBKP,2023-05-31 19:54:04.000,2023-05-31 19:00:00.000,MULTIPOINT ((-0.8887839089364267 -0.5156303040...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-31 18:00:00.000,0,SBKP,...,2023-05-31 19:00:00.000,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,None,None
97,c5e4e8d78906ae2b3218847cc9f045d9,SBSP,SBRJ,2023-05-20 15:16:05.000,2023-05-20 15:00:00.000,MULTIPOINT ((-0.8916331377478003 -0.5089690765...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-20 14:00:00.000,0,SBRJ,...,NaN,NaN,23.0,16.0,3.0,340.0,10000.0,1019.0,NaN,NaN
98,878ad3e9104c78f2945f6194c2d883ce,SBBR,SBSP,2023-05-31 11:31:02.000,2023-05-31 11:00:00.000,MULTIPOINT ((-0.8987280459174578 -0.5209708603...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-31 10:00:00.000,0,SBSP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [8]:
param_grid = {
    "eta": [0.05, 0.10, 0.15, 0.20],
    "max_depth": np.arange(3, 10, 1),
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4]
}

# Defina os parâmetros estáticos do modelo xgboost
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse"
}

# Cross-validation com busca de hiperparâmetros
grid = GridSearchCV(estimator=xgb.XGBRegressor(**params),
                    param_grid=param_grid,
                    cv=5,  # altere conforme a necessidade (3 é apenas um exemplo)
                    verbose=2,  # para mostrar a saída enquanto está ajustando
                    n_jobs=-1  # número de jobs (use todos os núcleos disponíveis se -1)
)

grid.fit(X_train, y_train)

# Mostra os melhores parâmetros encontrados
print("Best parameters found: ", grid.best_params_)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

[CV] END ...................eta=0.05, gamma=0.0, max_depth=3; total time=  20.8s
[CV] END ...................eta=0.05, gamma=0.0, max_depth=3; total time=  20.8s
[CV] END ...................eta=0.05, gamma=0.0, max_depth=3; total time=  20.9s
[CV] END ...................eta=0.05, gamma=0.0, max_depth=3; total time=  20.9s


/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

[CV] END ...................eta=0.05, gamma=0.0, max_depth=3; total time=  21.2s


...

[CV] END ....................eta=0.2, gamma=0.4, max_depth=9; total time=  34.2s


/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

[CV] END ....................eta=0.2, gamma=0.4, max_depth=9; total time=  30.8s


/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bruno/.local/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_catego

Best parameters found:  {'eta': 0.15, 'gamma': 0.0, 'max_depth': 5}


In [9]:

print("Best parameters found: ", grid.best_params_)

Best parameters found:  {'eta': 0.15, 'gamma': 0.0, 'max_depth': 5}


In [5]:
# Obtendo a importância dos recursos
feature_importance = model.get_score(importance_type='gain')

# Organizando os recursos por importância
sorted_feature_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# Imprimindo a importância dos recursos
for feature, importance in sorted_feature_importance:
    print(f"Feature: {feature}, Importance: {importance}")

Feature: distancia_mean, Importance: 67926794240.0
Feature: dt_radar_lambdamean, Importance: 23949686784.0
Feature: Pressãomean_x, Importance: 13577549824.0
Feature: flightlevel_meanmean, Importance: 8584696320.0
Feature: hora, Importance: 6759121920.0
Feature: diaSemana, Importance: 2819516928.0
Feature: troca_real_x, Importance: 2758203904.0
Feature: Ponto de Orvalhomean_x, Importance: 2404342784.0
Feature: trocareal, Importance: 2285466624.0
Feature: Temperaturamean_x, Importance: 2235195904.0
Feature: troca_real, Importance: 1791860608.0
Feature: Pressãomean_y, Importance: 1764953600.0
Feature: Direção do Ventostd_x, Importance: 1299741312.0
Feature: sum(flight_through_area), Importance: 1102850944.0
Feature: dt_radar_lambdamin, Importance: 1041450432.0
Feature: flightid_count, Importance: 974078848.0
Feature: troca, Importance: 963382272.0
Feature: mean_y, Importance: 929008000.0
Feature: esperas, Importance: 710263744.0
Feature: flightlevel_stdmean, Importance: 629546944.0
Featur

In [16]:
y_pred

array([4689.0093, 3817.7974, 2597.25  , ..., 7510.1313, 3047.6716,
       6835.697 ], dtype=float32)

In [29]:
for col in pd.read_csv("data/idsc_dataset.csv", delimiter=";").columns:
    print(col)

flightid
origem
destino
hora_ref
dt_dep
snapshot_radar
path
hora_esperas
esperas
aero_esperas
hora_metaf
metaf
aero_metaf
hora_metar
metar
aero_metar
hora_tcp
troca
aero_tcp
hora_tcr
aero_tcr
